In [1]:
using Statistics, LinearAlgebra, Random, DataFrames, Plots, StatsPlots, CSV
credits = CSV.read("credits.csv")
keywords = CSV.read("keywords.csv")
links = CSV.read("links.csv")
links_small = CSV.read("links_small.csv")
movies_metadata = CSV.read("movies_metadata.csv")
ratings = CSV.read("ratings.csv")
ratings_small = CSV.read("ratings_small.csv")

┌ Info: Recompiling stale cache file /Users/shiyixin/.julia/compiled/v1.2/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /Users/shiyixin/.julia/compiled/v1.2/StatsPlots/SiylL.ji for StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /Users/shiyixin/.julia/compiled/v1.2/CSV/HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1240


thread = 1 warning: only found 10 / 24 columns on data row: 19730. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 19731. Filling remaining columns with `missing`
thread = 1 warning: only found 10 / 24 columns on data row: 29503. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 29504. Filling remaining columns with `missing`
thread = 1 warning: only found 10 / 24 columns on data row: 35587. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 35588. Filling remaining columns with `missing`


,userId,movieId,rating,timestamp
,Int64,Int64,Float64,Int64
1,1,31,2.5,1260759144
2,1,1029,3.0,1260759179
3,1,1061,3.0,1260759182
4,1,1129,2.0,1260759185
5,1,1172,4.0,1260759205
6,1,1263,2.0,1260759151
7,1,1287,2.0,1260759187
8,1,1293,2.0,1260759148
9,1,1339,3.5,1260759125


In [6]:
names(movies_metadata)

24-element Array{Symbol,1}:
 :adult                
 :belongs_to_collection
 :budget               
 :genres               
 :homepage             
 :id                   
 :imdb_id              
 :original_language    
 :original_title       
 :overview             
 :popularity           
 :poster_path          
 :production_companies 
 :production_countries 
 :release_date         
 :revenue              
 :runtime              
 :spoken_languages     
 :status               
 :tagline              
 :title                
 :video                
 :vote_average         
 :vote_count           

In [104]:
movies_metadata = CSV.read("movies_metadata.csv")
budget = movies_metadata[:,:budget];

thread = 1 warning: only found 10 / 24 columns on data row: 19730. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 19731. Filling remaining columns with `missing`
thread = 1 warning: only found 10 / 24 columns on data row: 29503. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 29504. Filling remaining columns with `missing`
thread = 1 warning: only found 10 / 24 columns on data row: 35587. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 35588. Filling remaining columns with `missing`


In [122]:
Random.seed!(3)
randlist = filter(x -> x != "0", movies_metadata[:,:budget])
budget_rep = replace(budget, "/ff9qCepilowshEtG2GYWwzt2bs4.jpg"=>"0")
budget_rep = replace(budget_rep, "/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg"=>"0")
budget_rep = replace(budget_rep, "/zaSf5OG7V8X8gqFvly88zDdRm46.jpg"=>"0")
budget_rep = replace(budget_rep, "0"=>randlist[rand(1:num_uniq)], NaN=>randlist[rand(1:num_uniq)])
budget_repint = parse.(Float64, string.(budget_rep)) / 10^6
n = length(budget_repint)
budget_feature = zeros(n, 2)
for r in 1:n
    if budget_repint > 20
        budget_feature[r,:] = [1,1]
    elseif budget_repint > 1
        budget_feature[r,:] = [1,0]
    else
        budget_feature[r,:] = [0,0]
    end
end